# Optimize XG Boost - parameters / hyper parameters

In [39]:
import pandas as pd
import numpy as np
import researchpy as rp
from scipy import stats

import matplotlib.pyplot as plt
import pydotplus
from matplotlib import rcParams
import seaborn as sb


In [40]:
# # Find no. of factors in y
from collections import Counter
# #label encoding
from sklearn.preprocessing import LabelEncoder, scale, MinMaxScaler, StandardScaler,OneHotEncoder,LabelBinarizer
#from sklearn.decomposition import PCA

In [41]:
# Import All classifcation Algorithms
from xgboost import XGBClassifier

In [42]:
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score,validation_curve,KFold

In [43]:
# Metrics selection
from sklearn.metrics import  accuracy_score, recall_score, confusion_matrix,precision_recall_fscore_support,classification_report,roc_curve,auc

In [44]:
# Open files Both files are opened as ny drop or changes in feature will be done is both normal and one hot file
df1 = pd.read_csv('3_ITSM_cleansed_Encoded_Engineered_All.csv')
df1.columns

Index(['c_CI_Cat', 'c_CI_Name', 'c_CI_Subcat', 'c_Closure_Code', 'c_WBS',
       'q_No_of_Reassignments', 'q_No_of_Related_Incidents',
       'q_No_of_Related_Interactions', 't_Close_Time', 't_Open_Time',
       'y_Priority', 't_ReopenFlag', 't_TicketWIPDurationDays'],
      dtype='object')

In [45]:
CatVarList = ['c_CI_Cat', 'c_CI_Name', 'c_CI_Subcat', 'c_Closure_Code', 'c_WBS','t_ReopenFlag']
ConVarList = ['q_No_of_Reassignments', 'q_No_of_Related_Incidents','q_No_of_Related_Interactions','t_TicketWIPDurationDays']
DatVarList = ['t_Close_Time', 't_Open_Time']
OutValList = ['y_Priority']

X_All_1 = ['c_CI_Cat', 'c_CI_Name', 'c_CI_Subcat', 'c_Closure_Code', 'c_WBS','t_ReopenFlag',
           'q_No_of_Reassignments', 'q_No_of_Related_Incidents','q_No_of_Related_Interactions','t_TicketWIPDurationDays']
           

In [46]:
# Assign varibale and X and y
X1 = df1[X_All_1]
y1 = df1.y_Priority

print(X1.shape)
print(y1.shape)

(35846, 10)
(35846,)


In [47]:
# Split 80:20
split_seed = 10
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,random_state=split_seed, test_size=.2)
print(X1_train.shape)
print(y1_train.shape)
print(X1_test.shape)
print(y1_test.shape)

(28676, 10)
(28676,)
(7170, 10)
(7170,)


In [48]:
y1_train.head()

23572    4
163      3
30420    3
21679    3
23989    3
Name: y_Priority, dtype: int64

In [49]:
Counter(y1_train)

Counter({4: 6660, 3: 17679, 2: 3986, 1: 348, 0: 3})

## Scaling done based on EDA analysis. Fit on the Train set, but tranform both transform both training and test set

In [50]:
# Scale the data
scaler = StandardScaler()
# Fit on train data
scaler.fit(X1_train[ConVarList])
# Transform both train and test
X1_train[ConVarList] = scaler.transform(X1_train[ConVarList])
X1_test[ConVarList] = scaler.transform(X1_test[ConVarList])

E:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [51]:
print(X1_train.shape)
print(X1_test.shape)

(28676, 10)
(7170, 10)


## Grid Search to optimize the hyper parameters

In [52]:
# Parameters used for Grid search to get best parameters. Use this again only to get best parameters..its take a lot of time.
'''
parameters = {"learning_rate": [.05,.1, .15],
              "max_depth": [3,5,8,10,15,20],
              "min_child_weight" : [ 3, 5, 7 ],
              "gamma"            : [ 0.0, 0.1],
              "colsample_bytree" : [ 0.1,0.7,0.8], 
              "scoring"         : ["recall"] 
             }
'''
parameters = {"learning_rate": [.05],
              "max_depth": [15],
              "min_child_weight" : [5],
              "gamma"            : [0.1],
              "colsample_bytree" : [.7], 
              "scoring"         : ["recall"] 
             }

model = XGBClassifier(booster='gbtree',random_state=380)
grid_model_XGB = GridSearchCV(model, parameters)
grid_model_XGB.fit(X1_train, y1_train)

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=380, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.05], 'max_depth': [15], 'min_child_weight': [5], 'gamma': [0.1], 'colsample_bytree': [0.7], 'scoring': ['recall']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [53]:
grid_model_XGB.best_params_

{'colsample_bytree': 0.7,
 'gamma': 0.1,
 'learning_rate': 0.05,
 'max_depth': 15,
 'min_child_weight': 5,
 'scoring': 'recall'}

In [54]:
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test),columns=['y_Priority'])
print(accuracy_score(y1_test,y1_predict).round(4)*100)
#print(recall_score(y1_test,y1_predict).round(4)*100)
print(classification_report(y1_test,y1_predict))

83.63000000000001
              precision    recall  f1-score   support

           1       0.53      0.34      0.41        83
           2       0.83      0.66      0.73      1082
           3       0.84      0.94      0.89      4344
           4       0.83      0.71      0.77      1661

   micro avg       0.84      0.84      0.84      7170
   macro avg       0.76      0.66      0.70      7170
weighted avg       0.83      0.84      0.83      7170



In [55]:
print(y1_test.shape)
print(y1_predict.shape)

(7170,)
(7170, 1)


## Vary Model Random state to see if the results can be further improved

In [38]:
# masked after Running
'''
parameters = {"learning_rate": [.05],
              "max_depth": [15],
              "min_child_weight" : [5],
              "gamma"            : [0.1],
              "colsample_bytree" : [.7], 
              "scoring"         : ["recall"] 
             }

store = []
store_max=0
for i in range(10,500,5):
    model = XGBClassifier(booster='gbtree',random_state=i)
    grid_model_XGB = GridSearchCV(model, parameters)
    grid_model_XGB.fit(X1_train, y1_train)
    y1_predict = grid_model_XGB.predict(X1_test)
    if(accuracy_score(y1_test,y1_predict)>=store_max):
        store.append([i,accuracy_score(y1_test,y1_predict)*100])
        store_max=accuracy_score(y1_test,y1_predict)
    print(i,accuracy_score(y1_test,y1_predict)*100)
'''
# maxmim we find at random_state =90 @ 83.9

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


10 83.83542538354254


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


15 83.73779637377964


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


...

195 83.59832635983263


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


200 83.72384937238493


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

In [56]:
store

[[10, 83.83542538354254], [60, 83.83542538354254], [90, 83.90516039051604]]

## Final Model

In [57]:
parameters = {"learning_rate": [.05],
              "max_depth": [15],
              "min_child_weight" : [5],
              "gamma"            : [0.1],
              "colsample_bytree" : [.7], 
              "scoring"         : ["recall"] 
             }
model = XGBClassifier(booster='gbtree',random_state=90)
grid_model_XGB = GridSearchCV(model, parameters)
grid_model_XGB.fit(X1_train, y1_train)

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=90, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.05], 'max_depth': [15], 'min_child_weight': [5], 'gamma': [0.1], 'colsample_bytree': [0.7], 'scoring': ['recall']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [58]:
# Predicting and converting into Dataframe
#y1_predict = grid_model_XGB.predict(X1_test)
y1_predict = pd.DataFrame(grid_model_XGB.predict(X1_test))
#,columns=['y_Predict_Priority']
Proba = grid_model_XGB.predict_proba(X1_test)
y1_predict.shape
#y1_test.shape

(7170, 1)

In [59]:
# Results
print(accuracy_score(y1_test,y1_predict)*100)
#print(recall_score(y1_test,y1_predict)*100)
#print(classification_report(y1_test,y1_predict))
confusion_matrix(y1_test,y1_predict)
#Counter(y1_predict)

83.90516039051604


array([[  29,   18,   33,    3],
       [  15,  718,  302,   47],
       [  10,   78, 4069,  187],
       [   1,   46,  414, 1200]], dtype=int64)